In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [1]:
import openai
from openai import OpenAI

import concurrent.futures
import json
from pathlib import Path
import os

from virtual_lab.constants import CONSISTENT_TEMPERATURE, CREATIVE_TEMPERATURE
from virtual_lab.prompts import (
    CODING_RULES,
    REWRITE_PROMPT,
    create_merge_prompt
)
from virtual_lab.run_meeting import run_meeting
from virtual_lab.utils import load_summaries

from transcriptomics_constants import (
    background_prompt,
    experimental_results_prompt,
    num_iterations,
    num_rounds,
    discussions_phase_to_dir,
    principal_investigator,
    team_members,
)

## Team selection

In [2]:
# Team selection - prompts, pozor, když si jednou vyberu tým, už to znova nespouštím a nechám si ty konverzace v diskuzi
team_selection_agenda = f"""
{background_prompt}

{experimental_results_prompt}

You need to select a team of four scientists to help you with this transcriptomics project. The team should deal with these analytical challenges:

- Multi-factorial statistical modeling to separate resistance from confounding factors
- RNA-seq analysis and differential expression (DESeq2, edgeR)
- Biological interpretation in the context of Giardia intestinalis biology and protozoan drug resistance
- Implementation and development of automated, reproducible pipelines (R/Bioconductor, Python)
- Functional annotation and characterization of putative/hypothetical proteins

NOTE: Giardia intestinalis is a unique protozoan parasite with unusual biology. Understanding gene expression changes requires expertise in parasite physiology and drug resistance mechanisms.

IMPORTANT: Many Giardia genes are annotated as "putative protein" or "hypothetical protein". The team needs expertise in:
- Protein function prediction (sequence homology, domain analysis, structural prediction)
- Comparative genomics to infer function from related organisms
- Literature mining and database searches to assign putative functions

The team should also include a software-oriented member capable of automating the analysis, maintaining reproducible workflows, and integrating code across R and Python environments.
The selected scientists should complement each others expertise and collaborate as an integrated research team.

Please select the team members in the following format. You should NOT include yourself (Principal Investigator) in the list. Write the team as a Python list of Agent objects with "model=model" as the last parameter.

Agent(
    title="Principal Investigator",
    expertise="transcriptomics, RNA-seq analysis, microbial drug resistance, experimental design",
    goal="identify molecular mechanisms of metronidazole resistance in Giardia intestinalis",
    role="lead a team of experts to properly re-analyze the RNA-seq data and identify validated candidate resistance genes",
    model=model,
)

Principal Investigator, please provide your response.
"""


In [3]:
# Team selection - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_meeting,
            meeting_type="individual",
            team_member=principal_investigator,
            agenda=team_selection_agenda,
            save_dir=discussions_phase_to_dir["team_selection"],
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
        ) for iteration_num in range(num_iterations)
    ])

Rounds (+ Final Round):   0%|          | 0/1 [00:00<?, ?it/s]



































Team:   0%|          | 0/2 [00:10<?, ?it/s]





Rounds (+ Final Round): 100%|██████████| 1/1 [00:10<00:00, 10.73s/it]


Input token count: 1,175
Output token count: 413
Tool token count: 0
Max token length: 1,588
Cost: $0.01
Time: 0:13


Team:   0%|          | 0/2 [00:12<?, ?it/s]

Rounds (+ Final Round): 100%|██████████| 1/1 [00:12<00:00, 12.76s/it]


Input token count: 1,175
Output token count: 446
Tool token count: 0
Max token length: 1,621
Cost: $0.01
Time: 0:15


Team:   0%|          | 0/2 [00:13<?, ?it/s]







Rounds (+ Final Round): 100%|██████████| 1/1 [00:13<00:00, 13.59s/it]


Input token count: 1,175
Output token count: 708
Tool token count: 0
Max token length: 1,883
Cost: $0.01
Time: 0:16


Rounds (+ Final Round): 100%|██████████| 1/1 [00:15<00:00, 15.50s/it]


Input token count: 1,175
Output token count: 483
Tool token count: 0
Max token length: 1,658
Cost: $0.01
Time: 0:17


Team:   0%|          | 0/2 [00:15<?, ?it/s]



Rounds (+ Final Round): 100%|██████████| 1/1 [00:15<00:00, 15.70s/it]


Input token count: 1,175
Output token count: 476
Tool token count: 0
Max token length: 1,651
Cost: $0.01
Time: 0:18


In [4]:
# Team selection - merge
team_selection_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["team_selection"].glob("discussion_*.json")))
print(f"Number of summaries: {len(team_selection_summaries)}")

team_selection_merge_prompt = create_merge_prompt(agenda=team_selection_agenda)

run_meeting(
    meeting_type="individual",
    team_member=principal_investigator,
    summaries=team_selection_summaries,
    agenda=team_selection_merge_prompt,
    save_dir=discussions_phase_to_dir["team_selection"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
)

Number of summaries: 5


Rounds (+ Final Round): 100%|██████████| 1/1 [00:31<00:00, 31.99s/it]


Input token count: 3,851
Output token count: 924
Tool token count: 0
Max token length: 4,775
Cost: $0.02
Time: 0:33


## Projects specification

In [5]:
project_specification_agenda = f"""

{background_prompt}

{experimental_results_prompt} 
Design a transcriptomic analysis plan to identify genes specifically linked to metronidazole resistance in the BER line of Giardia intestinalis. Clearly distinguish resistance-specific effects from general drug responses and baseline strain differences. Propose a statistical model (e.g. with interaction terms) to detect these effects. Prioritize candidate resistance genes for validation and link them to biological functions using functional annotation tools. Include an approach for analyzing uncharacterized (putative) proteins."""

project_specification_questions = (
    "What is the most effective approach to identify genes linked to metronidazole resistance in *Giardia intestinalis*?",
    "How can resistance-specific expression be separated from general drug response and baseline differences between strains?",
    "Is a simple comparison sufficient, or is a complex statistical model needed? Why?",
    "How should candidate genes be functionally annotated and connected to biological pathways?",
    "What strategy can identify and characterize putative (unannotated) proteins among the candidate genes?",
    "Are any additional files, metadata, or annotations needed to perform the analysis effectively?",
)


In [11]:
for iteration_num in range(num_iterations):
    print(f"\n{'='*60}")
    print(f"🚀 Spouštím diskuzi {iteration_num + 1}/5...")
    print(f"{'='*60}\n")
    
    run_meeting(
        meeting_type="team",
        team_lead=principal_investigator,
        team_members=team_members,
        agenda=project_specification_agenda,
        agenda_questions=project_specification_questions,
        save_dir=discussions_phase_to_dir["project_specification"],
        save_name=f"discussion_{iteration_num + 1}",
        temperature=CREATIVE_TEMPERATURE,
        num_rounds=num_rounds,
    )
    
    print(f"\n✅ Diskuze {iteration_num + 1} dokončena!\n")


🚀 Spouštím diskuzi 1/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [06:34<00:00, 98.63s/it] 


Input token count: 75,320
Output token count: 6,773
Tool token count: 0
Max token length: 10,037
Cost: $0.26
Time: 6:39

✅ Diskuze 1 dokončena!


🚀 Spouštím diskuzi 2/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [04:47<00:00, 71.90s/it]


Input token count: 76,093
Output token count: 7,082
Tool token count: 0
Max token length: 10,346
Cost: $0.26
Time: 4:54

✅ Diskuze 2 dokončena!


🚀 Spouštím diskuzi 3/5...



...

Input token count: 75,598
Output token count: 6,910
Tool token count: 0
Max token length: 10,174
Cost: $0.26
Time: 6:01

✅ Diskuze 3 dokončena!


🚀 Spouštím diskuzi 4/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [05:32<00:00, 83.17s/it] 


Input token count: 78,545
Output token count: 7,269
Tool token count: 0
Max token length: 10,533
Cost: $0.27
Time: 5:37

✅ Diskuze 4 dokončena!


🚀 Spouštím diskuzi 5/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [05:27<00:00, 81.90s/it]


Input token count: 76,391
Output token count: 6,686
Tool token count: 0
Max token length: 9,950
Cost: $0.26
Time: 5:33

✅ Diskuze 5 dokončena!



In [12]:
# Project specification - merge
project_specification_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["project_specification"].glob("discussion_*.json")))
print(f"Number of summaries: {len(project_specification_summaries)}")

project_specification_merge_prompt = create_merge_prompt(
    agenda=project_specification_agenda,
    agenda_questions=project_specification_questions,
)

run_meeting(
    meeting_type="individual",
    team_member=principal_investigator,
    summaries=project_specification_summaries,
    agenda=project_specification_merge_prompt,
    save_dir=discussions_phase_to_dir["project_specification"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
    num_rounds=num_rounds,
)

Number of summaries: 5


Rounds (+ Final Round): 100%|██████████| 4/4 [02:54<00:00, 43.54s/it]


Input token count: 62,103
Output token count: 6,038
Tool token count: 0
Max token length: 12,618
Cost: $0.22
Time: 2:56


## Tools Selection

In [13]:
# Tools selection - prompts
tools_selection_agenda = f"""
{background_prompt}

{experimental_results_prompt}

Based on the finalized project specification, identify the specific computational and bioinformatics tools required to implement the transcriptomics analysis plan in practice.

Focus on selecting and justifying concrete tools, libraries, or platforms that will make the workflow executable and reproducible — from differential expression to functional annotation and visualization.

Your selection should:
- Build directly on the statistical and analytical strategy defined in the project specification.
- Include both established tools and recent innovations relevant to the field.
- Consider compatibility between R and Python environments (e.g., DESeq2 + Scikit-learn integration).
- Highlight new or improved AI-based platforms for protein structure and function prediction.
- Reflect the specific challenges of non-model organisms like Giardia intestinalis.

For each tool, please specify:
1. How it will be used in the workflow and at which step.
2. Why it is appropriate for this project and what makes it preferable to alternatives.
3. Any recent or experimental tools worth exploring as complementary options.
4. How the tool will interface with others in the pipeline to ensure automation and reproducibility.
"""


tools_selection_questions = (
    "Which computational tools and libraries should be used for each stage of the RNA-seq analysis (differential expression, annotation, visualization)?",
    "How will each selected tool be applied specifically to identify metronidazole resistance mechanisms?",
    "Which pathway databases and annotation resources are most appropriate for Giardia intestinalis?",
    "Which modern or AI-driven tools can improve the prediction and characterization of putative proteins?",
    "Are there any recent tools or databases developed for protozoan genomics that could enhance this analysis?",
)

tools_selection_prior_summaries = load_summaries(
    discussion_paths=[discussions_phase_to_dir["project_specification"] / "merged.json"])
print(f"Number of prior summaries: {len(tools_selection_prior_summaries)}")


Number of prior summaries: 1


In [14]:
for iteration_num in range(num_iterations):
    print(f"\n{'='*60}")
    print(f"🚀 Spouštím diskuzi {iteration_num + 1}/5...")
    print(f"{'='*60}\n")
    
    run_meeting(
        meeting_type="team",
        team_lead=principal_investigator,
        team_members=team_members,
        summaries=tools_selection_prior_summaries,
        agenda=tools_selection_agenda,
        agenda_questions=tools_selection_questions,
        save_dir=discussions_phase_to_dir["tools_selection"],
        save_name=f"discussion_{iteration_num + 1}",
        temperature=CREATIVE_TEMPERATURE,
        num_rounds=num_rounds,
    )
    
    print(f"\n✅ Diskuze {iteration_num + 1} dokončena!\n")


🚀 Spouštím diskuzi 1/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [06:01<00:00, 90.28s/it] 


Input token count: 102,836
Output token count: 7,779
Tool token count: 0
Max token length: 12,460
Cost: $0.33
Time: 6:05

✅ Diskuze 1 dokončena!


🚀 Spouštím diskuzi 2/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [06:16<00:00, 94.16s/it] 


Input token count: 107,298
Output token count: 8,105
Tool token count: 0
Max token length: 12,786
Cost: $0.35
Time: 6:22

✅ Diskuze 2 dokončena!


🚀 Spouštím diskuzi 3/5...



...

Input token count: 99,739
Output token count: 6,819
Tool token count: 0
Max token length: 11,500
Cost: $0.32
Time: 5:55

✅ Diskuze 3 dokončena!


🚀 Spouštím diskuzi 4/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [06:13<00:00, 93.48s/it] 


Input token count: 109,814
Output token count: 8,533
Tool token count: 0
Max token length: 13,214
Cost: $0.36
Time: 6:18

✅ Diskuze 4 dokončena!


🚀 Spouštím diskuzi 5/5...



Rounds (+ Final Round): 100%|██████████| 4/4 [06:05<00:00, 91.47s/it] 


Input token count: 108,758
Output token count: 8,300
Tool token count: 0
Max token length: 12,981
Cost: $0.35
Time: 6:10

✅ Diskuze 5 dokončena!



In [15]:
# Tools selection - merge
tools_selection_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["tools_selection"].glob("discussion_*.json")))
print(f"Number of summaries: {len(tools_selection_summaries)}")

tools_selection_merge_prompt = create_merge_prompt(
    agenda=tools_selection_agenda,
    agenda_questions=tools_selection_questions,
)

run_meeting(
    meeting_type="individual",
    team_member=principal_investigator,
    summaries=tools_selection_summaries,
    agenda=tools_selection_merge_prompt,
    save_dir=discussions_phase_to_dir["tools_selection"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
    num_rounds=num_rounds,
)

Number of summaries: 5


Rounds (+ Final Round): 100%|██████████| 4/4 [04:15<00:00, 63.92s/it]


Input token count: 63,594
Output token count: 6,760
Tool token count: 0
Max token length: 13,383
Cost: $0.23
Time: 4:18


## Implementation

In [16]:
# Implementation agent selection - prompts
implementation_agent_selection_agenda = f"""
{background_prompt}

{experimental_results_prompt}

Based on the project specification and selected tools, your team needs to implement the transcriptomics analysis pipeline.

The analysis requires implementation of 3 major components:

1. Statistical Analysis & QC
   - DESeq2 multi-factorial model
   - edgeR validation (optional cross-check)
   - Basic visualization (PCA, volcano plots)
   - Output: DEG lists, QC plots
   
2. Functional Analysis & Networks
   - ClusterProfiler (GO/KEGG/Reactome enrichment)
   - WGCNA (co-expression modules)
   - InterPro/Pfam (domain annotation)
   - STRING (protein interactions)
   - Candidate gene ranking
   - Output: Enriched pathways, gene modules, network data
   
3. AI-driven Characterization & Visualization
   - AlphaFold for unannotated proteins
   - DeepGOPlus function prediction
   - Comprehensive plots (heatmaps, networks, interactive)
   - Final integrated report
   - Output: Structure predictions, final figures


For each component, please select the team member who will implement the component. A team member may implement more than one component.

Consider each team member's expertise when making assignments.
"""

implementation_agent_selection_questions = (
    "Which team member will implement the statistical analysis pipeline (DESeq2, edgeR, QC visualization)?",
    "Which team member will implement the functional and network analysis pipeline (ClusterProfiler, WGCNA, STRING, InterPro)?",
    "Which team member will implement AI-driven characterization and comprehensive visualization (AlphaFold, DeepGOPlus, final plots)?",
)


implementation_agent_selection_prior_summaries = load_summaries(
    discussion_paths=[
        discussions_phase_to_dir["team_selection"] / "merged.json",
        discussions_phase_to_dir["project_specification"] / "merged.json",
        discussions_phase_to_dir["tools_selection"] / "merged.json"
    ]
)
print(f"Number of prior summaries: {len(implementation_agent_selection_prior_summaries)}")


Number of prior summaries: 3


In [17]:
# Implementation agent selection - discussion
for iteration_num in range(num_iterations):
    print(f"🚀 Spouštím implementation agent selection diskuzi {iteration_num + 1}/{num_iterations}...")
    run_meeting(
        meeting_type="individual",
        team_member=principal_investigator,
        summaries=implementation_agent_selection_prior_summaries,
        agenda=implementation_agent_selection_agenda,
        agenda_questions=implementation_agent_selection_questions,
        save_dir=discussions_phase_to_dir["implementation_agent_selection"],
        save_name=f"discussion_{iteration_num + 1}",
        temperature=CREATIVE_TEMPERATURE,
    )
    print(f"✅ Diskuze {iteration_num + 1} dokončena!")


🚀 Spouštím implementation agent selection diskuzi 1/5...


Rounds (+ Final Round): 100%|██████████| 1/1 [00:21<00:00, 21.06s/it]


Input token count: 4,762
Output token count: 596
Tool token count: 0
Max token length: 5,358
Cost: $0.02
Time: 0:22
✅ Diskuze 1 dokončena!
🚀 Spouštím implementation agent selection diskuzi 2/5...


Rounds (+ Final Round): 100%|██████████| 1/1 [00:25<00:00, 25.82s/it]


Input token count: 4,762
Output token count: 561
Tool token count: 0
Max token length: 5,323
Cost: $0.02
Time: 0:29
✅ Diskuze 2 dokončena!
🚀 Spouštím implementation agent selection diskuzi 3/5...


...

Input token count: 4,762
Output token count: 483
Tool token count: 0
Max token length: 5,245
Cost: $0.02
Time: 0:34
✅ Diskuze 3 dokončena!
🚀 Spouštím implementation agent selection diskuzi 4/5...


Rounds (+ Final Round): 100%|██████████| 1/1 [00:19<00:00, 19.99s/it]


Input token count: 4,762
Output token count: 454
Tool token count: 0
Max token length: 5,216
Cost: $0.02
Time: 0:21
✅ Diskuze 4 dokončena!
🚀 Spouštím implementation agent selection diskuzi 5/5...


Rounds (+ Final Round): 100%|██████████| 1/1 [00:12<00:00, 12.67s/it]


Input token count: 4,762
Output token count: 442
Tool token count: 0
Max token length: 5,204
Cost: $0.02
Time: 0:14
✅ Diskuze 5 dokončena!


In [18]:
# Implementation - merge
implementation_agent_selection_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["implementation_agent_selection"].glob("discussion_*.json")))
print(f"Number of summaries: {len(implementation_agent_selection_summaries)}")

implementation_agent_selection_merge_prompt = create_merge_prompt(
    agenda=implementation_agent_selection_agenda,
    agenda_questions=implementation_agent_selection_questions
)

run_meeting(
    meeting_type="individual",
    team_member=principal_investigator,
    summaries=implementation_agent_selection_summaries,
    agenda=implementation_agent_selection_merge_prompt,
    save_dir=discussions_phase_to_dir["implementation_agent_selection"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
)

Number of summaries: 5


Rounds (+ Final Round): 100%|██████████| 1/1 [00:17<00:00, 17.44s/it]


Input token count: 3,810
Output token count: 671
Tool token count: 0
Max token length: 4,481
Cost: $0.02
Time: 0:19


In [ ]:
# ============================================================
# Quick team sanity check: review of task distribution
# ============================================================
from virtual_lab.prompts import SCIENTIFIC_CRITIC

implementation_agent_selection_team_prompt = """
The Principal Investigator has assigned implementation tasks among the team members.
Please discuss whether this task distribution is realistic and well-balanced.
Focus on whether any team member might be overloaded or if the division could be simplified.
Do not propose new methods — only assess the practicality and balance of the current plan.
"""

print("Starting short team sanity check...")

run_meeting(
    meeting_type="team",
    team_lead=principal_investigator,
    team_members=team_members + (SCIENTIFIC_CRITIC,),
    summaries=implementation_agent_selection_summaries,
    agenda=implementation_agent_selection_team_prompt,
    save_dir=discussions_phase_to_dir["implementation_agent_selection"],
    save_name="team_sanity_check",
    temperature=CONSISTENT_TEMPERATURE,
)
print("Team sanity check completed.")


Starting short team sanity check...


Rounds (+ Final Round): 100%|██████████| 1/1 [00:13<00:00, 13.21s/it]


Input token count: 2,888
Output token count: 246
Tool token count: 0
Max token length: 3,134
Cost: $0.01
Time: 0:19
Team sanity check completed.


## Workflow Design

In [ ]:
# Workflow design - prompts
workflow_design_agenda = f"""
{background_prompt}

{experimental_results_prompt}

Based on the project specification, selected tools, and implementation assignments, design a detailed step-by-step workflow for the transcriptomics analysis.

The workflow should cover:
1. Data preparation and quality control
2. Statistical analysis (DESeq2 multi-factorial model)
3. Functional annotation (pathway enrichment)
4. Putative protein characterization
5. Visualization and reporting

Provide a clear, modular, and reproducible workflow with inputs, outputs, and quality checks for each step.
"""

workflow_design_questions = (
    "What is the complete step-by-step workflow for the analysis?",
    "What are the inputs and outputs for each major step?",
    "What quality control checks should be performed at each stage?",
    "How will the different analysis components integrate together?",
    "What are the key decision points and how should they be handled?",
)

workflow_design_prior_summaries = load_summaries(
    discussion_paths=[
        discussions_phase_to_dir["team_selection"] / "merged.json",
        discussions_phase_to_dir["project_specification"] / "merged.json",
        discussions_phase_to_dir["tools_selection"] / "merged.json",
        discussions_phase_to_dir["implementation_agent_selection"] / "merged.json"
    ]
)
print(f"Number of prior summaries: {len(workflow_design_prior_summaries)}")



In [ ]:
# Workflow design - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_meeting,
            meeting_type="individual",
            team_member=principal_investigator,
            agenda=workflow_design_agenda,
            agenda_questions=workflow_design_questions,
            save_dir=discussions_phase_to_dir["workflow_design"],
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Workflow design - merge
workflow_design_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["workflow_design"].glob("discussion_*.json")))
print(f"Number of summaries: {len(workflow_design_summaries)}")

workflow_design_merge_prompt = create_merge_prompt(
    agenda=workflow_design_agenda,
    agenda_questions=workflow_design_questions,
)

run_meeting(
    meeting_type="individual",
    team_member=principal_investigator,
    summaries=workflow_design_summaries,
    agenda=workflow_design_merge_prompt,
    save_dir=discussions_phase_to_dir["workflow_design"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
)

## Writting Scripts

In [ ]:
# ========================================
# IMPLEMENTATION PHASES
# ========================================

# DESeq2 Statistical Analysis - prompts
deseq2_analysis_agenda = f"""
{background_prompt}

{experimental_results_prompt}

You are the Bioinformatics Statistician. Based on the workflow design, write a COMPLETE R script that implements the statistical analysis pipeline.

The script must:
1. Load count matrix (experimental_data/A2_count_matrix.txt) and sample metadata
2. Create DESeq2 object with multi-factorial design: ~ genotype + treatment + genotype:treatment
3. Filter low-count genes (≥10 counts in ≥3 samples)
4. Run DESeq2 normalization and analysis
5. Define contrasts to isolate resistance-specific effects
6. Extract significant genes (FDR < 0.05, |log2FC| > 1.5)
7. Perform diagnostic checks
8. Export results to CSV files
9. Include clear comments

Write the complete R script now.
"""

deseq2_analysis_prior_summaries = load_summaries(
    discussion_paths=[
        discussions_phase_to_dir["workflow_design"] / "merged.json",
        discussions_phase_to_dir["tools_selection"] / "merged.json"
    ]
)
print(f"Number of prior summaries: {len(deseq2_analysis_prior_summaries)}")


In [ ]:
# DESeq2 Statistical Analysis - implementation
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_meeting,
            meeting_type="individual",
            team_member=team_members[1],  # Bioinformatics Statistician
            summaries=deseq2_analysis_prior_summaries,
            agenda=deseq2_analysis_agenda,
            save_dir=discussions_phase_to_dir["deseq2_analysis"],
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CONSISTENT_TEMPERATURE,
        ) for iteration_num in range(num_iterations)
    ])


In [ ]:
# DESeq2 Statistical Analysis - merge
deseq2_analysis_summaries = load_summaries(
    discussion_paths=list(discussions_phase_to_dir["deseq2_analysis"].glob("discussion_*.json"))
)
print(f"Number of summaries: {len(deseq2_analysis_summaries)}")

deseq2_analysis_merge_prompt = create_merge_prompt(agenda=deseq2_analysis_agenda)

run_meeting(
    meeting_type="individual",
    team_member=team_members[1],  # Bioinformatics Statistician
    summaries=deseq2_analysis_summaries,
    agenda=deseq2_analysis_merge_prompt,
    save_dir=discussions_phase_to_dir["deseq2_analysis"],
    save_name="merged",
    temperature=CONSISTENT_TEMPERATURE,
)


## Virtual Lab Analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rcParams.update({'font.size': 26})

In [ ]:
figure_dir = Path("figures/virtual_lab_analysis")
figure_dir.mkdir(parents=True, exist_ok=True)

phase_to_agent_to_word_count = {}

In [ ]:
# Count words that the human user wrote
phase_to_human_words = {
    "team_selection": [
        background_prompt,
        principal_investigator.prompt,
        scientific_critic.prompt,
        team_selection_agenda.replace(f"{background_prompt} ", ""),
    ],
    "project_specification": [
        project_specification_agenda.replace(f"{background_prompt} ", ""),
        *project_specification_questions,
        nanobody_prompt,
    ],
    "tools_selection": [
        tools_selection_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        *tools_selection_questions,
    ],
    "implementation_agent_selection": [
        implementation_agent_selection_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        *implementation_agent_selection_questions,
    ],
    "esm": [
        esm_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        improve_esm_agenda.replace(f" {REWRITE_PROMPT}", ""),
    ],
    "alphafold": [
        alphafold_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        improve_alphafold_agenda.replace(f" {REWRITE_PROMPT}", ""),
    ],
    "rosetta": [
        rosetta_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        improve_rosetta_xml_agenda.replace(f" {REWRITE_PROMPT}", ""),
        improve_rosetta_python_agenda.replace(f" {REWRITE_PROMPT}", ""),
    ],
    "workflow_design": [
        workflow_design_agenda.replace(f"{background_prompt} {nanobody_prompt} ", ""),
        *workflow_design_questions,
    ],
}

for phase, human_words in phase_to_human_words.items():
    phase_to_agent_to_word_count[phase] = {"Human Researcher": len(" ".join(human_words).split())}

In [ ]:
# Count words that the LLM agents wrote
for phase_name in ["team_selection", "project_specification", "tools_selection",
                   "implementation_agent_selection", "esm", "alphafold", "rosetta", "workflow_design"]:
    phase_dir = discussions_phase_to_dir[phase_name]

    print(f"Phase: {phase_name}")

    # Load the text written by each agent
    agent_to_text = {}
    for path in phase_dir.glob("*.json"):
        with open(path) as f:
            discussion = json.load(f)

        for message in discussion:
            agent_to_text.setdefault(message["agent"], []).append(message["message"])

    # Count the number of words written by each agent
    for agent, text in agent_to_text.items():
        if agent == "User":
            continue

        agent_to_text[agent] = " ".join(text)
        word_count = len(agent_to_text[agent].split())
        phase_to_agent_to_word_count[phase_name][agent] = word_count

# Print words by phase
for phase in phase_to_agent_to_word_count:
    print(f"Phase: {phase}")
    for agent, word_count in phase_to_agent_to_word_count[phase].items():
        print(f"Number of words written by {agent}: {word_count:,}")
    print()

# Sum word counts across phases
agent_to_word_count = {}
for phase in phase_to_agent_to_word_count:
    for agent, word_count in phase_to_agent_to_word_count[phase].items():
        agent_to_word_count[agent] = agent_to_word_count.get(agent, 0) + word_count

# Total number of words written by each LLM agent
for agent, word_count in agent_to_word_count.items():
    print(f"Total number of words written by {agent}: {word_count:,}")

print()

# Total number of words written by all LLM agents
total_human_words = sum(
    phase_to_agent_to_word_count[phase]["Human Researcher"] for phase in phase_to_agent_to_word_count)
total_agent_words = sum(word_count for agent, word_count in agent_to_word_count.items() if agent != "Human Researcher")

print(f"Total number of words written by Human Researcher: {total_human_words:,}")
print(f"Total number of words written by all LLM agents: {total_agent_words:,}")

In [ ]:
agent_to_color = {
    agent: sns.color_palette("tab10", n_colors=len(agent_to_word_count))[i]
    for i, agent in enumerate(agent_to_word_count)
}

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.pie(
    agent_to_word_count.values(),
    labels=agent_to_word_count.keys(),
    autopct="%1.1f%%",
    colors=[agent_to_color[agent] for agent in agent_to_word_count],
)
ax.set_title(f"Words written")
plt.savefig(figure_dir / "total_words_written.pdf", bbox_inches="tight")

In [ ]:
for phase in phase_to_agent_to_word_count:
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.pie(
        phase_to_agent_to_word_count[phase].values(),
        labels=phase_to_agent_to_word_count[phase].keys(),
        autopct="%1.1f%%",
        colors=[agent_to_color[agent] for agent in phase_to_agent_to_word_count[phase]],
    )
    ax.set_title(f"Words written in {phase.replace('_', ' ')}")
    plt.savefig(figure_dir / f"{phase}_words_written.pdf", bbox_inches="tight")